In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve

## Neighbourhood wise modelling

In [7]:
def onehot_encode(X, cols):
    # Treat new categories as a new 'unknown' category (all onehot columns are 0)
    onehot_enc = OneHotEncoder(handle_unknown='ignore')
    # Fit encoder on training data
    onehot_enc.fit(X[cols])
    # Get the names of the new columns created
    colnames = list(onehot_enc.get_feature_names(input_features=cols))
    # Transform the data
    onehot_vals = onehot_enc.transform(X[cols]).toarray()
    # Put transformed data into dataframe
    enc_df = pd.DataFrame(onehot_vals,columns=colnames,index=X.index)
    # Add onehot columns back onto original dataframe and drop the original columns
    X = pd.concat([X,enc_df],axis=1).drop(cols,axis=1)
    return X,onehot_enc

In [8]:
## Read the data for modelling
df = pd.read_csv('data/final_data_for_modelling.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106980 entries, 0 to 106979
Data columns (total 26 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   gender                                      106980 non-null  object 
 1   age                                         106980 non-null  int64  
 2   neighbourhood                               106980 non-null  object 
 3   scholarship                                 106980 non-null  int64  
 4   hypertension                                106980 non-null  int64  
 5   diabetes                                    106980 non-null  int64  
 6   alcoholism                                  106980 non-null  int64  
 7   handicap                                    106980 non-null  int64  
 8   sms_received                                106980 non-null  int64  
 9   showed                                      106980 non-null  int64  
 

In [9]:
## get a list of neighbourhoods
neigh_list = list(df.neighbourhood.unique())
filter_neigh_list = [neigh for neigh in neigh_list if len(df[df["neighbourhood"] == neigh])>2000]

filter_neigh_list

['JARDIM DA PENHA',
 'ANDORINHAS',
 'NOVA PALESTINA',
 'DA PENHA',
 'TABUAZEIRO',
 'SÃO PEDRO',
 'SANTA MARTHA',
 'SANTO ANDRÉ',
 'BONFIM',
 'JARDIM CAMBURI',
 'MARIA ORTIZ',
 'JABOUR',
 'RESISTÊNCIA',
 'SANTO ANTÔNIO',
 'ITARARÉ',
 'CENTRO',
 'CARATOÍRA',
 'JESUS DE NAZARETH',
 'ILHA DO PRÍNCIPE',
 'ROMÃO']

In [10]:
## Make dataframe filters for each neighbourhood
data = {}
for neigh in filter_neigh_list:
    df_temp = df.copy()
    df_temp = df_temp[df_temp["neighbourhood"] == neigh]
    df_temp.drop(columns=["age_group", "Neighbourhood", "neighbourhood", "showed"] , inplace=True)
    data[neigh] = df_temp

In [11]:
## Create one model for each neighbourhood

for key, datafr in data.items():
    # Splitting dataset into test and train
    X_train, X_test, y_train, y_test = train_test_split(datafr.drop(columns=["no_show"]), datafr["no_show"], random_state=0,test_size=0.2)

    ## Ordinal encoder for features
    enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

    ## Fit encoder on train and apply to test data as well
    X_train[["gender"]] = enc.fit_transform(X_train[["gender"]])
    X_test[["gender"]] = enc.transform(X_test[["gender"]])

    ## One hot encode the train data
    cols = ["appointment_day_of_week"]
    X_train, onehot_enc = onehot_encode(X_train, cols)

    # Apply onehot encoder to test data
    colnames = columns=list(onehot_enc.get_feature_names(input_features=cols))
    onehot_vals = onehot_enc.transform(X_test[cols]).toarray()

    # Put transformed data into dataframe
    enc_df = pd.DataFrame(onehot_vals,columns=colnames,index=X_test.index)
    # Add onehot columns back onto original dataframe and drop the original columns
    X_test = pd.concat([X_test,enc_df],axis=1).drop(cols, axis=1)

    # Create the parameter grid based on the results of random search 
    param_grid = {
        'max_depth' : [3,4,5,6],
        'min_samples_leaf': [2, 3, 4, 5],
        'n_estimators': [25, 50, 75],
        'random_state':[0],
        'criterion' :['gini', 'entropy'],
        'class_weight': [{1:4}]
    }

    # Create a based model
    rf = RandomForestClassifier()
    # Instantiate the grid search model
    grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                                cv = 3, n_jobs = -1, verbose = 0, scoring='precision')

    # Fit the grid search to the data
    grid_search.fit(X_train, y_train)

    ## Train the best model
    best_model = grid_search.best_estimator_
    best_model.fit(X_train, y_train)

    ## Classsification report
    y_pred_train = best_model.predict(X_train)
    print(f"Train Report: {key}")
    print(classification_report(y_train, y_pred_train))
    y_pred_test = best_model.predict(X_test)
    print("Test Report")
    print(classification_report(y_test, y_pred_test))


/Users/architkaila/opt/anaconda3/envs/aipi520/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/architkaila/opt/anaconda3/envs/aipi520/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Train Report: JARDIM DA PENHA
              precision    recall  f1-score   support

           0       0.87      0.89      0.88      2590
           1       0.33      0.30      0.31       490

    accuracy                           0.79      3080
   macro avg       0.60      0.59      0.60      3080
weighted avg       0.78      0.79      0.79      3080

Test Report
              precision    recall  f1-score   support

           0       0.84      0.85      0.85       634
           1       0.26      0.24      0.25       137

    accuracy                           0.75       771
   macro avg       0.55      0.55      0.55       771
weighted avg       0.74      0.75      0.74       771



/Users/architkaila/opt/anaconda3/envs/aipi520/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/architkaila/opt/anaconda3/envs/aipi520/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Train Report: ANDORINHAS
              precision    recall  f1-score   support

           0       0.96      0.51      0.67      1335
           1       0.38      0.94      0.54       421

    accuracy                           0.62      1756
   macro avg       0.67      0.73      0.60      1756
weighted avg       0.82      0.62      0.64      1756

Test Report
              precision    recall  f1-score   support

           0       0.94      0.51      0.66       351
           1       0.31      0.86      0.45        88

    accuracy                           0.58       439
   macro avg       0.62      0.69      0.56       439
weighted avg       0.81      0.58      0.62       439



/Users/architkaila/opt/anaconda3/envs/aipi520/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/architkaila/opt/anaconda3/envs/aipi520/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


...

Train Report: JESUS DE NAZARETH
              precision    recall  f1-score   support

           0       0.96      0.60      0.74      1633
           1       0.43      0.93      0.59       542

    accuracy                           0.68      2175
   macro avg       0.70      0.76      0.67      2175
weighted avg       0.83      0.68      0.70      2175

Test Report
              precision    recall  f1-score   support

           0       0.93      0.58      0.71       410
           1       0.40      0.87      0.55       134

    accuracy                           0.65       544
   macro avg       0.66      0.72      0.63       544
weighted avg       0.80      0.65      0.67       544



/Users/architkaila/opt/anaconda3/envs/aipi520/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/architkaila/opt/anaconda3/envs/aipi520/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Train Report: ILHA DO PRÍNCIPE
              precision    recall  f1-score   support

           0       0.96      0.58      0.72      1354
           1       0.41      0.91      0.56       423

    accuracy                           0.66      1777
   macro avg       0.68      0.75      0.64      1777
weighted avg       0.82      0.66      0.68      1777

Test Report
              precision    recall  f1-score   support

           0       0.92      0.57      0.70       343
           1       0.36      0.83      0.51       102

    accuracy                           0.63       445
   macro avg       0.64      0.70      0.61       445
weighted avg       0.79      0.63      0.66       445



/Users/architkaila/opt/anaconda3/envs/aipi520/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/architkaila/opt/anaconda3/envs/aipi520/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Train Report: ROMÃO
              precision    recall  f1-score   support

           0       0.95      0.48      0.63      1329
           1       0.33      0.91      0.48       371

    accuracy                           0.57      1700
   macro avg       0.64      0.69      0.56      1700
weighted avg       0.81      0.57      0.60      1700

Test Report
              precision    recall  f1-score   support

           0       0.94      0.46      0.62       331
           1       0.32      0.91      0.48        95

    accuracy                           0.56       426
   macro avg       0.63      0.68      0.55       426
weighted avg       0.81      0.56      0.58       426

